In [3]:
import pandas as pd
from openai import OpenAI
import json
import re
from pydantic import BaseModel
import time
from tqdm import tqdm
import os
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from pydantic import BaseModel
import os
import openai
import os
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score
from datasets import load_dataset
import pandas as pd
from openai import OpenAI
import json
import re
from pydantic import BaseModel
import time
from tqdm import tqdm
import os
import requests
import json
import os
import getpass
from tqdm import tqdm
import pandas as pd
import time
import requests, json, os, time
from tqdm.notebook import tqdm
import pandas as pd
# Cell for implementing the recursive reasoning summarizer loop
import pandas as pd
import openai
import json
import re
import time
import requests
import random
from tqdm.notebook import tqdm

In [11]:
df = pd.read_csv('data/aime/aime_2024_completions_updated.csv')
df

,ID,Problem,Solution,Answer,model_completion,extracted_answer
0,2024-II-4,"Let $x,y$ and $z$ be positive real numbers tha...","Denote $\log_2(x) = a$, $\log_2(y) = b$, and $...",33,"<think>\nOkay, so I have this problem where I ...",33
1,2024-II-12,"Let $O(0,0), A(\tfrac{1}{2}, 0),$ and $B(0, \t...",Begin by finding the equation of the line $\ov...,23,"<think>\nOkay, so I need to solve this geometr...",23
2,2024-I-4,Jen enters a lottery by picking $4$ distinct n...,This is a conditional probability problem. Bay...,116,"<think>\nOkay, let's try to figure out this pr...",116
3,2024-I-3,Alice and Bob play the following game. A stack...,Let's first try some experimentation. Alice ob...,809,"<think>\nOkay, so Alice and Bob are playing th...",809
4,2024-I-8,Eight circles of radius $34$ are sequentially ...,Draw an altitude from both end circles of the ...,197,"<think>\nAlright, so I need to solve this geom...",2926
5,2024-I-12,Define $f(x)=|| x|-\tfrac{1}{2}|$ and $g(x)=||...,We will denote $h(x)=4g(f(x))$ for simplicity....,385,"<think>\nOkay, I need to solve this problem wh...",16
6,2024-I-11,Each vertex of a regular octagon is independen...,Let $r$ be the number of red vertices and $b$ ...,371,"<think>\nAlright, let me try to figure out thi...",279
7,2024-II-11,Find the number of triples of nonnegative inte...,$ab(a+b)+bc(b+c)+ac(a+c)=300(ab+bc+ac)-3abc=60...,601,"<think>\nOkay, so I need to find the number of...",601
8,2024-I-2,"There exist real numbers $x$ and $y$, both gre...","By properties of logarithms, we can simplify t...",25,"<think>\nOkay, let me try to figure out this p...",25
9,2024-II-6,Alice chooses a set $A$ of positive integers. ...,Let $k$ be one of the elements in Alice's set ...,55,"<think>\nOkay, let's try to figure out this pr...",55


In [ ]:
# Function to summarize reasoning trace using GPT-4o
def summarize_reasoning(reasoning_trace):
    """
    Summarize the model's reasoning trace into a more concise form while preserving critical reasoning steps.
    """
    openai_client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    summarizer_prompt = [
        {"role": "system", "content": """You are an expert at summarizing mathematical reasoning traces.
        Your goal is to produce concise but complete summaries of reasoning traces that:
        1. Preserve all key reasoning steps and intermediate calculations
        2. Highlight potential errors or invalid assumptions in the original reasoning
        3. Identify what paths of reasoning were explored and abandoned
        4. Maintain the logical flow and dependencies between steps
        5. Avoid introducing new reasoning or calculations not present in the original trace
        6. Format the summary in a clear, step-by-step manner
        7. Include key equations and numerical results with proper notation
        
        Your summary should be detailed enough that another model could understand the full reasoning process,
        but concise enough to eliminate redundancy and verbose explanations."""},
        {"role": "user", "content": f"""Below is a reasoning trace from a math problem. 
        Summarize this reasoning trace while preserving all key steps, intermediate calculations, 
        and potential errors in the original reasoning. Return just the summary.
        
        REASONING TRACE:
        {reasoning_trace}
        
        SUMMARY:"""}
    ]
    
    for attempt in range(3):
        try:
            response = openai_client.chat.completions.create(
                model="gpt-4o",
                messages=summarizer_prompt,
                temperature=0
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Summarization attempt {attempt+1} failed: {e}")
            if attempt < 2:
                print(f"Retrying in {(2 ** attempt) * 2} seconds...")
                time.sleep((2 ** attempt) * 2)
            else:
                print("Max retries reached. Returning original reasoning.")
                return reasoning_trace

# Function to get continued reasoning from DeepSeek-R1
def continue_reasoning(question, reasoning_so_far, fireworks_key, model="accounts/fireworks/models/deepseek-r1", max_tokens=20000):
    """
    Continue reasoning by submitting a new completion request with the summarized reasoning
    """
    # Generate a random continuation phrase
    continuation_phrases = [
        "Let me continue thinking more deeply about this problem.",
        "I need to explore this problem further to find the correct solution.",
        "Let me reconsider my approach and continue reasoning.",
        "I should take a different angle and continue my reasoning.",
        "I'll extend my reasoning to ensure I arrive at the correct answer.",
        "I need to think about this more carefully."
    ]
    continuation_phrase = random.choice(continuation_phrases)
    
    # Construct prompt with original question, summarized reasoning outside think tags
    prompt = f"Solve the following question: \n\n'{question}'\n\nHere's a summary of previous reasoning attempts:\n{reasoning_so_far}\n\nWe have strong reason to believe we need to reconsider. Let's think more carefully, learning from our summary.\n\n<think>"
    
    # API constants
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {fireworks_key}"
    }
    url = "https://api.fireworks.ai/inference/v1/completions"
    
    # Make API call with retry logic
    for attempt in range(3):
        try:
            response = requests.post(
                url, 
                headers=headers,
                json={
                    "model": model,
                    "max_tokens": max_tokens,
                    "temperature": 0.7,
                    "prompt": prompt
                },
            )
            
            if response.status_code == 200:
                new_completion = response.json().get("choices", [{}])[0].get("text", "")
                # Combine with think tag
                combined_reasoning = f"<think>{new_completion}"
                return combined_reasoning, prompt
            elif response.status_code == 429:  # Rate limit
                wait_time = (2 ** attempt) * 2
                print(f"Rate limited. Waiting {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"API error: {response.status_code}, {response.text}")
                if attempt == 2:
                    return reasoning_so_far  # Return original reasoning if all attempts fail
        except Exception as e:
            print(f"Error: {str(e)}")
            time.sleep(2)
            if attempt == 2:
                return reasoning_so_far, f"Solve the following question: \n\n'{question}'\n\n<think>"  # Return original reasoning if all attempts fail

# Function to extract reasoning from between <think> and </think> tags
def extract_thinking(completion):
    """Extract the reasoning part between <think> and </think> tags"""
    if pd.isna(completion):
        return ""
    
    # Find content between <think> and </think>
    pattern = r'<think>(.*?)</think>'
    match = re.search(pattern, completion, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        # If no closing tag, take everything after <think>
        start_idx = completion.find("<think>")
        if start_idx != -1:
            return completion[start_idx + 7:].strip()
        else:
            return completion.strip()

# Extract answer from boxed notation
def extract_boxed_answer(text):
    """Extract answer from \boxed{X} notation"""
    if pd.isna(text):
        return ""
    
    pattern = r'\\boxed\{([^}]+)\}'
    match = re.search(pattern, text)
    
    if match:
        return match.group(1).strip()
    else:
        return ""

# Main function to run the recursive reasoning loop with checkpointing
def run_recursive_reasoning_loop(df, fireworks_key, answer_column='Answer', max_iterations=2, checkpoint_path='data/aime/reasoning_checkpoint.csv'):
    """
    Process each row, check if extracted_answer differs from answer, and run reasoning summarizer loop
    with checkpointing after each processed row and iteration
    
    Args:
        df: DataFrame with model_completion and extracted_answer columns
        fireworks_key: API key for Fireworks
        answer_column: Column name containing the correct answer
        max_iterations: Maximum iterations of summarize-continue loop
        checkpoint_path: Path to save checkpoint files
        
    Returns:
        Updated DataFrame with new_completion and new_extracted_answer columns
    """
    # Make sure required columns exist
    if 'extracted_answer' not in df.columns:
        print("Adding extracted_answer column...")
        df['extracted_answer'] = df['model_completion'].apply(extract_boxed_answer)
    
    # Add new columns if they don't exist
    for col in ['summarized_completion', 'summarized_extracted_answer', 'iterations', 
                'summarized_reasoning_trace', 'new_prompt', 'new_completion', 'new_extracted_answer']:
        if col not in df.columns:
            df[col] = None
    
    # Initialize iterations column with 0 if it doesn't have values
    df['iterations'] = df['iterations'].fillna(0).astype(int)
    
    # Create a checkpoint directory if it doesn't exist
    checkpoint_dir = os.path.dirname(checkpoint_path)
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Load existing checkpoint if it exists
    if os.path.exists(checkpoint_path):
        print(f"Loading checkpoint from {checkpoint_path}")
        try:
            checkpoint_df = pd.read_csv(checkpoint_path)
            # Update only processed rows from checkpoint
            for idx in checkpoint_df.index:
                if idx < len(df) and checkpoint_df.loc[idx, 'iterations'] > 0:
                    for col in ['summarized_reasoning_trace', 'new_prompt', 'new_completion', 
                                'new_extracted_answer', 'iterations']:
                        df.loc[idx, col] = checkpoint_df.loc[idx, col]
            print(f"Loaded checkpoint with {(df['iterations'] > 0).sum()} processed rows")
        except Exception as e:
            print(f"Error loading checkpoint: {e}")
    
    # Find rows with mismatched answers
    mismatch_indices = []
    for idx in range(len(df)):
        extracted = str(df.loc[idx, 'extracted_answer']).strip() if not pd.isna(df.loc[idx, 'extracted_answer']) else ""
        answer = str(df.loc[idx, answer_column]).strip() if not pd.isna(df.loc[idx, answer_column]) else ""
        
        # Only include rows that haven't been fully processed (iterations < max_iterations)
        if extracted != answer and not pd.isna(df.loc[idx, 'model_completion']) and df.loc[idx, 'iterations'] < max_iterations:
            mismatch_indices.append(idx)
    
    print(f"Found {len(mismatch_indices)} rows with mismatched answers to process")
    
    # Process each mismatched row
    for row_num, idx in enumerate(mismatch_indices):
        print(f"\nProcessing row {idx} ({row_num+1}/{len(mismatch_indices)})")
        question = df.loc[idx, 'Problem']
        original_completion = df.loc[idx, 'model_completion']
        
        # Extract the reasoning from the original completion
        reasoning = extract_thinking(original_completion)
        
        if not reasoning:
            print(f"No reasoning found for row {idx}, skipping")
            continue
        
        # Current reasoning starts with original reasoning or previous iteration
        if df.loc[idx, 'iterations'] > 0 and not pd.isna(df.loc[idx, 'new_completion']):
            print(f"Continuing from previous iteration {df.loc[idx, 'iterations']}")
            current_reasoning = extract_thinking(df.loc[idx, 'new_completion'])
            current_completion = df.loc[idx, 'new_completion']
            start_iteration = int(df.loc[idx, 'iterations'])
        else:
            current_reasoning = reasoning
            current_completion = original_completion
            start_iteration = 0
        
        # Track if we found a boxed answer in any iteration
        found_boxed_answer = False
        
        # Run up to max_iterations of summarize-continue loop
        for iteration in range(start_iteration, max_iterations):
            print(f"\nRow {idx}, Iteration {iteration+1}:")
            
            # Step 1: Summarize the reasoning
            print("Summarizing reasoning...")
            summarized_reasoning = summarize_reasoning(current_reasoning)

            # Store the summarized reasoning in the DataFrame
            df.loc[idx, 'summarized_reasoning_trace'] = summarized_reasoning
            print(f"Summarized reasoning length: {len(summarized_reasoning)} chars")
            
            # Step 2: Continue reasoning with DeepSeek-R1
            print("Continuing reasoning with DeepSeek-R1...")
            new_completion, new_prompt = continue_reasoning(question, summarized_reasoning, fireworks_key)
            
            # Extract the new reasoning
            current_reasoning = extract_thinking(new_completion)
            current_completion = new_completion
            
            # Update the DataFrame with the current iteration results
            df.loc[idx, 'new_prompt'] = new_prompt
            df.loc[idx, 'new_completion'] = current_completion
            df.loc[idx, 'new_extracted_answer'] = extract_boxed_answer(current_completion)
            df.loc[idx, 'iterations'] = iteration + 1
            
            # Save checkpoint after each iteration
            checkpoint_filename = f"{os.path.splitext(checkpoint_path)[0]}_{row_num}_{iteration+1}.csv"
            df.to_csv(checkpoint_filename, index=False)
            df.to_csv(checkpoint_path, index=False)
            print(f"Saved checkpoint to {checkpoint_filename}")
            
            # Check if we now have a boxed answer
            new_extracted = extract_boxed_answer(current_completion)
            
            if new_extracted:
                found_boxed_answer = True
                print(f"Found answer: {new_extracted}")
                break
        
        print(f"\nFinal completion for row {idx}: Answer = {df.loc[idx, 'new_extracted_answer']}")
        
        # Save main checkpoint after each row
        df.to_csv(checkpoint_path, index=False)
        print(f"Saved main checkpoint after processing row {idx}")
    
    # Calculate statistics
    fixed_count = 0
    for idx in range(len(df)):
        if df.loc[idx, 'iterations'] > 0:  # Only check rows we processed
            extracted = str(df.loc[idx, 'extracted_answer']).strip() if not pd.isna(df.loc[idx, 'extracted_answer']) else ""
            new_extracted = str(df.loc[idx, 'new_extracted_answer']).strip() if not pd.isna(df.loc[idx, 'new_extracted_answer']) else ""
            answer = str(df.loc[idx, answer_column]).strip() if not pd.isna(df.loc[idx, answer_column]) else ""
            
            if extracted != answer and new_extracted == answer:
                fixed_count += 1
    
    total_processed = (df['iterations'] > 0).sum()
    if total_processed > 0:
        print(f"\nResolved {fixed_count} out of {total_processed} processed answers ({fixed_count/total_processed*100:.2f}%)")
    
    # Save final results
    final_path = f"{os.path.splitext(checkpoint_path)[0]}_final.csv"
    df.to_csv(final_path, index=False)
    print(f"Saved final results to {final_path}")
    
    return df

# Run the recursive reasoning loop with checkpoint saving
if 'fireworks_key' not in globals():
    fireworks_key = os.environ.get("FIREWORKS_API_KEY")
    if not fireworks_key:
        fireworks_key = input("Enter your Fireworks API key: ")

# Define the checkpoint path
checkpoint_path = 'data/aime/reasoning_checkpoint.csv'

# Run the process with checkpointing
df_updated = run_recursive_reasoning_loop(
    df, 
    fireworks_key, 
    answer_column='Answer', 
    max_iterations=2,
    checkpoint_path=checkpoint_path
)

print("Complete! Results saved.")

Found 7 rows with mismatched answers to process

Processing row 4 (1/7)
Continuing from previous iteration 1

Row 4, Iteration 2:
Summarizing reasoning...
Summarized reasoning length: 1836 chars
Continuing reasoning with DeepSeek-R1...
Saved checkpoint to data/aime/reasoning_checkpoint_0_2.csv
Found answer: 197

Final completion for row 4: Answer = 197
Saved main checkpoint after processing row 4

Processing row 5 (2/7)
Continuing from previous iteration 1

Row 5, Iteration 2:
Summarizing reasoning...
Summarized reasoning length: 1790 chars
Continuing reasoning with DeepSeek-R1...
Saved checkpoint to data/aime/reasoning_checkpoint_1_2.csv
Found answer: 16

Final completion for row 5: Answer = 16
Saved main checkpoint after processing row 5

Processing row 6 (3/7)
Continuing from previous iteration 1

Row 6, Iteration 2:
Summarizing reasoning...
Summarized reasoning length: 1950 chars
Continuing reasoning with DeepSeek-R1...
Saved checkpoint to data/aime/reasoning_checkpoint_2_2.csv
Fou

In [4]:
df

,ID,Problem,Solution,Answer,model_completion,extracted_answer,summarized_completion,summarized_extracted_answer,iterations,new_completion,new_extracted_answer
0,2024-II-4,"Let $x,y$ and $z$ be positive real numbers tha...","Denote $\log_2(x) = a$, $\log_2(y) = b$, and $...",33,"<think>\nOkay, so I have this problem where I ...",33,None,None,0,NaN,NaN
1,2024-II-12,"Let $O(0,0), A(\tfrac{1}{2}, 0),$ and $B(0, \t...",Begin by finding the equation of the line $\ov...,23,"<think>\nOkay, so I need to solve this geometr...",23,None,None,0,NaN,NaN
2,2024-I-4,Jen enters a lottery by picking $4$ distinct n...,This is a conditional probability problem. Bay...,116,"<think>\nOkay, let's try to figure out this pr...",116,None,None,0,NaN,NaN
3,2024-I-3,Alice and Bob play the following game. A stack...,Let's first try some experimentation. Alice ob...,809,"<think>\nOkay, so Alice and Bob are playing th...",809,None,None,0,NaN,NaN
4,2024-I-8,Eight circles of radius $34$ are sequentially ...,Draw an altitude from both end circles of the ...,197,"<think>\nAlright, so I need to solve this geom...",2926,None,None,1,<think>\n1. **Problem Understanding**: The pro...,2927
5,2024-I-12,Define $f(x)=|| x|-\tfrac{1}{2}|$ and $g(x)=||...,We will denote $h(x)=4g(f(x))$ for simplicity....,385,"<think>\nOkay, I need to solve this problem wh...",16,None,None,1,<think>\n1. **Problem Setup**: The task is to ...,16
6,2024-I-11,Each vertex of a regular octagon is independen...,Let $r$ be the number of red vertices and $b$ ...,371,"<think>\nAlright, let me try to figure out thi...",279,None,None,1,<think>\n1. **Problem Understanding**: The tas...,279
7,2024-II-11,Find the number of triples of nonnegative inte...,$ab(a+b)+bc(b+c)+ac(a+c)=300(ab+bc+ac)-3abc=60...,601,"<think>\nOkay, so I need to find the number of...",601,None,None,0,NaN,NaN
8,2024-I-2,"There exist real numbers $x$ and $y$, both gre...","By properties of logarithms, we can simplify t...",25,"<think>\nOkay, let me try to figure out this p...",25,None,None,0,NaN,NaN
9,2024-II-6,Alice chooses a set $A$ of positive integers. ...,Let $k$ be one of the elements in Alice's set ...,55,"<think>\nOkay, let's try to figure out this pr...",55,None,None,0,NaN,NaN


In [8]:
df_updated = pd.read_csv('data/aime/recursive_reasoning_results.csv').dropna(axis = 1, how='all')

In [9]:
df_updated

,ID,Problem,Solution,Answer,model_completion,extracted_answer,iterations,new_completion,new_extracted_answer
0,2024-II-4,"Let $x,y$ and $z$ be positive real numbers tha...","Denote $\log_2(x) = a$, $\log_2(y) = b$, and $...",33,"<think>\nOkay, so I have this problem where I ...",33,0,NaN,NaN
1,2024-II-12,"Let $O(0,0), A(\tfrac{1}{2}, 0),$ and $B(0, \t...",Begin by finding the equation of the line $\ov...,23,"<think>\nOkay, so I need to solve this geometr...",23,0,NaN,NaN
2,2024-I-4,Jen enters a lottery by picking $4$ distinct n...,This is a conditional probability problem. Bay...,116,"<think>\nOkay, let's try to figure out this pr...",116,0,NaN,NaN
3,2024-I-3,Alice and Bob play the following game. A stack...,Let's first try some experimentation. Alice ob...,809,"<think>\nOkay, so Alice and Bob are playing th...",809,0,NaN,NaN
4,2024-I-8,Eight circles of radius $34$ are sequentially ...,Draw an altitude from both end circles of the ...,197,"<think>\nAlright, so I need to solve this geom...",2926,1,<think>\n1. **Problem Understanding**: The pro...,2927.0
5,2024-I-12,Define $f(x)=|| x|-\tfrac{1}{2}|$ and $g(x)=||...,We will denote $h(x)=4g(f(x))$ for simplicity....,385,"<think>\nOkay, I need to solve this problem wh...",16,1,<think>\n1. **Problem Setup**: The task is to ...,16.0
6,2024-I-11,Each vertex of a regular octagon is independen...,Let $r$ be the number of red vertices and $b$ ...,371,"<think>\nAlright, let me try to figure out thi...",279,1,<think>\n1. **Problem Understanding**: The tas...,279.0
7,2024-II-11,Find the number of triples of nonnegative inte...,$ab(a+b)+bc(b+c)+ac(a+c)=300(ab+bc+ac)-3abc=60...,601,"<think>\nOkay, so I need to find the number of...",601,0,NaN,NaN
8,2024-I-2,"There exist real numbers $x$ and $y$, both gre...","By properties of logarithms, we can simplify t...",25,"<think>\nOkay, let me try to figure out this p...",25,0,NaN,NaN
9,2024-II-6,Alice chooses a set $A$ of positive integers. ...,Let $k$ be one of the elements in Alice's set ...,55,"<think>\nOkay, let's try to figure out this pr...",55,0,NaN,NaN
